## Flow for paper modelling different PPA scenarios

Outline:
1. Basic data analysis: quality of data, basic statistics. For both load and gen
2. Select the scenario to be modelled. Variables to define include:
    * PPA delivery structure (PaP, shaped, baseload)
    * Use of (and params around) demand shifting
    * Whether a battery is added and what size
3. Create the 'optimal' hybrid profile according to the delivery structure:
    * For PaP - 'optimal' is best match to customer load
    * For shaped - 'optimal' is best match to customer load (maybe? Or is this just most consistent shape, i.e. most reliable to deliver? Probably this but idk how this would work)
    * For baseload - 'optimal' is best match to contract shape
4. If load shifting and/or battery operation is involved: add that here
5. Compare traces to find key values:
    * Matched and unmatched load
    * Matched and unmatched contracted generation
    * Wholesale value of each match/unmatch load and gen
    * Emissions associated with unmatched load
6. Calculate reasonable strike price(s) including risk-based premiums. Decide and define firming contract
7. Run ppa.calc scripts including firming contract details to receive financial outcomes
8. See what happens!

In [1]:
# ------------------------------ Packages & Files ------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import nemed
import ppa, residuals, tariffs, hybrid
import calendar
import holidays
import pprint
from mip import Model, xsum, minimize, CONTINUOUS, BINARY, conflict
from nemosis import dynamic_data_compiler, static_table
from typing import List
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from getting_data import import_gen_data, import_load_data, import_pricing_data, import_emissions_data
from helper_functions import _check_interval_consistency, _check_missing_data, get_interval_length

pd.set_option('display.max_rows', None)

INFO: Using Python-MIP package version 1.15.0


In [2]:
# -------------------------------- USER INPUTS ---------------------------------

# - - - - - - - - - - - - - - - - - - DATA - - - - - - - - - - - - - - - - - - -

# load data file name
LOAD_FN = '/Users/elliekallmier/Desktop/RA_Work/247_TRUZERO/247_ppa/ppa_analysis/tests/test_file.csv'
LOAD_COL_NAME = 'Load'
LOAD_TIMEZONE = ''

# generation data file name, or, generator DUID(s)
# TODO: add function to take input TYPE of generator and select appropriate trace
GEN_FN = ''
GEN_COL_NAME_S = ''

# get name of datetime column for both load and generation files
LOAD_DATETIME_COL_NAME = 'TS'
GEN_DATETIME_COL_NAME = ''
DAY_FIRST = True
GEN_TECH_TYPE_S = ['WIND - ONSHORE', 'PHOTOVOLTAIC FLAT PANEL']

# Region to fetch generation/pricing/emissions data for:
REGION = 'QLD1'

# NEMOSIS inputs:
RAW_DATA_CACHE = 'data_caches/gen_data_cache'
EMISSIONS_CACHE = 'data_caches/nemed_cache'
PRICING_CACHE = 'data_caches/pricing_cache'


# - - - - - - - - - - - - - - - - - CONTRACT - - - - - - - - - - - - - - - - - -
# TODO: fill out
DELIVERY_STRUCTURE = ''
FIRMING_CONTRACT_TYPE = ''

EXPOSURE_BOUND_UPPER = ''
EXPOSURE_BOUND_LOWER = '' 

FLOOR_PRICE = ''



In [3]:
# -------------------------------- Get Load Data -------------------------------
#   - check dtypes of columns - should all be float, except datetime col.
#   - update colname(s)
#   - set datetime index
#   - get interval length
#   - check for NaN/missing data

load_data, interval, start_date, end_date = import_load_data.get_load_data(LOAD_FN, LOAD_DATETIME_COL_NAME, LOAD_COL_NAME, DAY_FIRST)

Some missing data found. Filled with zeros.



In [4]:
# ----------------------------- Get Generation Data ----------------------------
gen_data = import_gen_data.get_generation_data(RAW_DATA_CACHE, REGION, GEN_TECH_TYPE_S, interval, start_date=start_date, end_date=end_date)

INFO: Retrieving static table Generators and Scheduled Loads
INFO: Downloading data for table Generators and Scheduled Loads
INFO: Compiling data for table DISPATCH_UNIT_SCADA
INFO: Returning DISPATCH_UNIT_SCADA.
Some missing data found. Filled with zeros.



In [5]:
gen_data.head(1)

,CSPVPS1: Photovoltaic Flat Panel,COOPGWF1: Wind - Onshore,DDSF1: Photovoltaic Flat Panel,KEPWF1: Wind - Onshore,KSP1: Photovoltaic Flat panel,MEWF1: Wind - Onshore,SMCSF1: Photovoltaic Flat panel,SRSF1: Photovoltaic Flat panel,WARWSF1: Photovoltaic Flat Panel,WARWSF2: Photovoltaic Flat Panel
DateTime,,,,,,,,,,
2022-01-01,0.0,286.062,0.0,9.878,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# ------------------------ Get Pricing & Emissions Data ------------------------
emissions_intensity = import_emissions_data.get_avg_emissions_intensity(
    start_date, end_date, EMISSIONS_CACHE, [REGION], period=f'{interval}min'
)

price_data = import_pricing_data.get_wholesale_price_data(
    start_date, end_date, PRICING_CACHE, [REGION], period=f'{interval}min'
)

INFO: Processing total emissions from 2021-12-31 to 2022-01-01
INFO: Compiling data for table DISPATCH_UNIT_SCADA
INFO: Returning DISPATCH_UNIT_SCADA.
INFO: Compiling Energy from Dispatch
INFO: Processing total emissions from 2022-01-01 to 2022-01-31
INFO: Compiling data for table DISPATCH_UNIT_SCADA
INFO: Returning DISPATCH_UNIT_SCADA.
INFO: Compiling Energy from Dispatch
INFO: Loading results file processed_co2_total_2021-12-31_2022-01-01.parquet
INFO: Loading results file processed_co2_total_2022-01-01_2022-01-31.parquet
INFO: Completed get_total_emissions_by_DI_DUID
INFO: Compiling data for table DISPATCHPRICE
INFO: Returning DISPATCHPRICE.


In [7]:
price_data.head(1)

REGIONID,RRP: QLD1
DateTime,
2022-01-01,116.675116


In [ ]:
# Now combine all of the data together:

combined_data = pd.concat([load_data, gen_data, price_data, emissions_intensity], axis='columns')